In [1]:
# Importing Libraries
# Import required libraries
 
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn import tree
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr

In [2]:
#Loading Data as csv File
# Load and preprocess the data
data = pd.read_csv('MTN Data.csv')
data = data.dropna()  # Remove missing values
X = data.drop(['YEAR'], axis=1)
X = data[['SN', 'TAG', 'MONTH']]
y = data['GC']

In [3]:
data.size

273875

In [4]:
data.head()

,SN,TAG,MONTH,YEAR,GC
0,1,62040,12,2018,303
1,2,71812,12,2018,607
2,3,73466,12,2018,150
3,4,90540,12,2018,141
4,5,92071,12,2018,154


In [5]:
# Data Description
data.size
print(data.shape)
data.describe().transpose()

(54775, 5)


,count,mean,std,min,25%,50%,75%,max
SN,54775.0,27388.000000,15812.324834,1.0,13694.5,27388.0,41081.5,54775.0
TAG,54775.0,62344.021908,23190.864090,33.0,53297.0,63112.0,77675.0,165786.0
MONTH,54775.0,6.120000,3.290258,1.0,3.0,6.0,9.0,12.0
YEAR,54775.0,2020.240000,1.335078,2018.0,2019.0,2020.0,2021.0,2023.0
GC,54775.0,128.936431,191.328045,1.0,31.0,90.0,151.0,2921.0


In [6]:
# Splitting the Data into Training (80%) & Testing Dataset (20%)
# Split the data into training and testing sets
train_size = int(0.8 * len(data))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [7]:
# Training Individual Models
# Train the individual models
gbm = GradientBoostingRegressor()
rf = RandomForestRegressor()
 
gbm.fit(X_train, y_train)
rf.fit(X_train, y_train)

RandomForestRegressor()

In [8]:
# Stacking method
y_pred_gbm = gbm.predict(X_test)
y_pred_rf = rf.predict(X_test)
X_test_combined = pd.concat([pd.Series(y_pred_gbm), pd.Series(y_pred_rf)], axis=1)
hybrid_model = GradientBoostingRegressor()
hybrid_model.fit(X_test_combined, y_test)

GradientBoostingRegressor()

In [9]:
# Performance Evaluation for Hybrid Model
# Evaluate the performance of the hybrid model
y_pred_hybrid = hybrid_model.predict(X_test_combined)
rmse_hybrid = mean_squared_error(y_test, y_pred_hybrid, squared=False)
r2 = r2_score(y_test, y_pred_hybrid)
print("RMSE of Hybrid Model:", rmse_hybrid)
print("R2 Score of Hybrid Model:",r2)

RMSE of Hybrid Model: 84.60255261893968
R2 Score of Hybrid Model: 0.13433157086422154


In [10]:
# Performance Comparison with other Models
# Compare the performance of the hybrid model with the individual models
y_pred_gbm_train = gbm.predict(X_train)
y_pred_rf_train = rf.predict(X_train)
X_train_combined = pd.concat([pd.Series(y_pred_gbm_train), pd.Series(y_pred_rf_train)], axis=1)
hybrid_model_train = GradientBoostingRegressor()
hybrid_model_train.fit(X_train_combined, y_train)
y_pred_hybrid_train = hybrid_model_train.predict(X_train_combined)
rmse_gbm = mean_squared_error(y_train, y_pred_gbm_train, squared=False)
rmse_rf = mean_squared_error(y_train, y_pred_rf_train, squared=False)
rmse_hybrid_train = mean_squared_error(y_train, y_pred_hybrid_train, squared=False)
print("RMSE of Gradient Boosting Model:", rmse_gbm)
print("RMSE of Random Forest Model:", rmse_rf)
print("RMSE of Hybrid Model (Training):", rmse_hybrid_train)

RMSE of Gradient Boosting Model: 197.2150815715549
RMSE of Random Forest Model: 83.14726837973612
RMSE of Hybrid Model (Training): 58.175280521053075


In [12]:
# Gradio interface
def stacking_model(sn, tag, month):
    # Create a dataframe with the input values
    input_data = pd.DataFrame({
        'SN': [sn],
        'TAG': [tag],
        'MONTH': [month]
    })

    # Make predictions with the individual models
    gbm_pred = gbm.predict(input_data)[0]
    rf_pred = rf.predict(input_data)[0]

    # Combine predictions for stacking
    combined_pred = pd.Series([gbm_pred, rf_pred]).values.reshape(1, -1)

    # Predict with the hybrid model
    hybrid_pred = hybrid_model.predict(combined_pred)[0]

    return int(hybrid_pred)

iface = gr.Interface(
    fn=stacking_model,
    inputs=[
        "number", "number", "number"
    ],
    outputs="number",
    live=True,
    title="Hybrid Model Stacking Regressor Predictor",
    description="Predict GC using stacking method with Gradient Boosting and Random Forest"
)

# Launch the Gradio interface
iface.launch()

Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.
